In [74]:
import pandas as pd
import numpy as np
import plotly.express as px


In [75]:
# Load the RNA-seq data
rnaseq_df = pd.read_csv('/Users/johnsmith/Desktop/Senior Design Stuff/tcga.brca.rnaseq.unstranded.fpkm.counts.matrix (1).txt', sep='\t', index_col=0)

# Load the HRD score data
hrd_df = pd.read_excel('/Users/johnsmith/Desktop/Senior Design Stuff/tcga.hrdscore.xlsx')

# Load the BRCA status data
brca_df = pd.read_csv('/Users/johnsmith/Desktop/Senior Design Stuff/toga.breast.brca.status.txt', sep='\t', index_col=0)
brca_df.index = brca_df.index.str.replace('.', '-')


In [76]:
#idenfity the common samples

# Extract sample IDs from each dataframe
rnaseq_samples = set(rnaseq_df['Case ID'])
hrd_samples = set(hrd_df['sample'])
brca_samples = set(brca_df.index)

# Find the intersection of all sample IDs
common_samples = rnaseq_samples.intersection(hrd_samples).intersection(brca_samples)

# Print the number of common samples
print("\nNumber of common samples:", len(common_samples))


Number of common samples: 864


In [77]:
# Drop samples that are not common in all three dataframes
rnaseq_df_common = rnaseq_df[rnaseq_df['Case ID'].isin(common_samples)]
hrd_df_common = hrd_df[hrd_df['sample'].isin(common_samples)]
brca_df_common = brca_df[brca_df.index.isin(common_samples)]

# Print the shapes of the new dataframes
print("rnaseq_df_common:", rnaseq_df_common.shape)
print("hrd_df_common:", hrd_df_common.shape)
print("brca_df_common:", brca_df_common.shape)

rnaseq_df_common: (929, 60663)
hrd_df_common: (864, 5)
brca_df_common: (864, 33)


In [78]:
#merge metadata

metadata = pd.merge(hrd_df_common, brca_df_common, left_on='sample', right_index=True)
metadata.set_index('sample', inplace=True)

print (metadata.shape)
print(metadata.columns)
metadata.head()

(864, 37)
Index(['HRD', 'Telomeric AI', 'LST', 'HRD-sum', 'BRCA1_somatic_null',
       'BRCA1_germ_bi_allelic', 'BRCA1_germ_mono_allelic', 'BRCA1_deletion',
       'BRCA1_epigenetic_silencing', 'BRCA1_mRNA', 'BRCA2_somatic_null',
       'BRCA2_germ_bi_allelic', 'BRCA2_germ_undetermined',
       'BRCA2_germ_mono_allelic', 'BRCA2_deletion', 'RAD51C_germ',
       'RAD51C_deletion', 'RAD51C_epigenetic_silencing', 'RAD51C_mRNA',
       'PALB2_somatic_null', 'PALB2_germ', 'TP53_somatic', 'H1', 'H2', 'H3',
       'H4', 'H1.norm', 'H2.norm', 'H3.norm', 'H4.norm', 'event.BRCA1',
       'event.BRCA2', 'event.RAD51C', 'event.PALB2', 'event.All Events',
       'event.PAM50', 'event.TNBC'],
      dtype='object')


,HRD,Telomeric AI,LST,HRD-sum,BRCA1_somatic_null,BRCA1_germ_bi_allelic,BRCA1_germ_mono_allelic,BRCA1_deletion,BRCA1_epigenetic_silencing,BRCA1_mRNA,...,H2.norm,H3.norm,H4.norm,event.BRCA1,event.BRCA2,event.RAD51C,event.PALB2,event.All Events,event.PAM50,event.TNBC
sample,,,,,,,,,,,,,,,,,,,,,
TCGA-A1-A0SB,0,0,0,0,0,0,0,0,0,0,...,0.044837,0.635070,0.000000,0,0,0,0,0,NaN,0
TCGA-A1-A0SD,10,10,7,27,0,0,0,0,0,0,...,0.254169,0.703974,0.041857,0,0,0,0,0,LumA,0
TCGA-A1-A0SE,6,6,1,13,0,0,0,0,0,0,...,0.102998,0.740053,0.120368,0,0,0,0,0,LumA,0
TCGA-A1-A0SF,7,11,8,26,0,0,0,0,0,0,...,0.000000,0.395329,0.604671,0,0,0,0,0,LumA,0
TCGA-A1-A0SG,3,2,4,9,0,0,0,0,0,0,...,0.183277,0.776319,0.040404,0,0,0,0,0,LumA,0


In [79]:

# Start with the original dataframe
metadata_filtered = metadata.copy()

# Drop rows where event.BRCA1 is '1'
metadata_filtered = metadata_filtered[metadata_filtered['event.BRCA1'] != '1']

# Drop rows where event.BRCA2 is 'Bi-allelic-undetermined'
metadata_filtered = metadata_filtered[metadata_filtered['event.BRCA2'] != 'Bi-allelic-undetermined']

# Drop rows where event.PALB2 is '2'
metadata_filtered = metadata_filtered[metadata_filtered['event.PALB2'] != '2']

print (metadata_filtered.shape)
print(metadata_filtered.columns)
metadata_filtered.head()

metadata = metadata_filtered
metadata['HRD_status_base'] = ['HRD' if x >= 42 else 'HR' for x in metadata['HRD-sum']]


(857, 37)
Index(['HRD', 'Telomeric AI', 'LST', 'HRD-sum', 'BRCA1_somatic_null',
       'BRCA1_germ_bi_allelic', 'BRCA1_germ_mono_allelic', 'BRCA1_deletion',
       'BRCA1_epigenetic_silencing', 'BRCA1_mRNA', 'BRCA2_somatic_null',
       'BRCA2_germ_bi_allelic', 'BRCA2_germ_undetermined',
       'BRCA2_germ_mono_allelic', 'BRCA2_deletion', 'RAD51C_germ',
       'RAD51C_deletion', 'RAD51C_epigenetic_silencing', 'RAD51C_mRNA',
       'PALB2_somatic_null', 'PALB2_germ', 'TP53_somatic', 'H1', 'H2', 'H3',
       'H4', 'H1.norm', 'H2.norm', 'H3.norm', 'H4.norm', 'event.BRCA1',
       'event.BRCA2', 'event.RAD51C', 'event.PALB2', 'event.All Events',
       'event.PAM50', 'event.TNBC'],
      dtype='object')


In [82]:
px.box(metadata, x = 'event.PAM50', y = 'HRD-sum', color='HRD_status_base')

In [83]:
def softLabel(x, median, HRD_thresh, HRP_thresh):
    adjustment = 2 * ((((HRD_thresh - x) / (HRD_thresh - HRP_thresh)) - 0.5) ** 2) + 0.5
    if median < x < HRD_thresh:
        return adjustment
    elif HRP_thresh <= x < median:
        return 1 - adjustment
    else:
        return -1


In [84]:
medians = metadata.groupby(["event.PAM50"])["HRD-sum"].median().reset_index() 
medians_pivot = medians.set_index('event.PAM50').T
medians_pivot

event.PAM50,Basal,Her2,LumA,LumB
HRD-sum,55.0,35.0,13.0,31.0


In [85]:
for sub in medians_pivot.columns.tolist():
    metadata['HRD_adjust'] = ['HRD' if x > 50 else 'HRP' if x < 10 else 'HRD ambiguous' if medians_pivot.loc['HRD-sum',sub] < x <50 else 'HRP ambiguous' if 10 < x < medians_pivot.loc['HRD-sum',sub] else 'None' for x in metadata['HRD-sum']]
    metadata['soft prob'] = metadata['HRD-sum'].apply(lambda x: softLabel(x, medians_pivot.loc['HRD-sum',sub], 50, 10))

In [86]:
metadata

,HRD,Telomeric AI,LST,HRD-sum,BRCA1_somatic_null,BRCA1_germ_bi_allelic,BRCA1_germ_mono_allelic,BRCA1_deletion,BRCA1_epigenetic_silencing,BRCA1_mRNA,...,event.BRCA1,event.BRCA2,event.RAD51C,event.PALB2,event.All Events,event.PAM50,event.TNBC,HRD_status_base,HRD_adjust,soft prob
sample,,,,,,,,,,,,,,,,,,,,,
TCGA-A1-A0SB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,NaN,0,HR,HRP,-1.00000
TCGA-A1-A0SD,10,10,7,27,0,0,0,0,0,0,...,0,0,0,0,0,LumA,0,HR,HRP ambiguous,0.48875
TCGA-A1-A0SE,6,6,1,13,0,0,0,0,0,0,...,0,0,0,0,0,LumA,0,HR,HRP ambiguous,0.13875
TCGA-A1-A0SF,7,11,8,26,0,0,0,0,0,0,...,0,0,0,0,0,LumA,0,HR,HRP ambiguous,0.48000
TCGA-A1-A0SG,3,2,4,9,0,0,0,0,0,0,...,0,0,0,0,0,LumA,0,HR,HRP,-1.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-OL-A6VR,4,4,5,13,0,0,0,NaN,0,0,...,0,0,0,0,0,LumA,0,HR,HRP ambiguous,0.13875
TCGA-PE-A5DC,13,20,16,49,0,0,0,0,NaN,0,...,0,Bi-allelic-inactivation,0,0,YES,LumA,0,HRD,HRD ambiguous,0.95125
TCGA-PE-A5DD,7,6,9,22,0,0,0,0,NaN,0,...,0,0,0,0,0,LumA,0,HR,HRP ambiguous,0.42000


In [87]:
metadata_dropna = metadata[metadata['HRD_adjust'] != 'None']

In [89]:
px.box(metadata_dropna, x = 'event.PAM50', y = 'HRD-sum', color = 'HRD_adjust')